In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.6 MB/s eta 0:00:00


# загрузим данные

https://www.kaggle.com/coolonce/recipes-and-interpretation-dim

In [2]:
!ls

sample_data


In [3]:
import pandas as pd

In [5]:
df_rec = pd.read_csv('/content/all_recepies_inter.csv', sep='\t')


In [6]:
df_rec[:4]

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."


In [7]:
print(df_rec.loc[0, 'Инструкции'])

Подготовить указанные ингредиенты для приготовления рассольника с перловой крупой. Мясной бульон сварить заранее из говядины или из курицы, также можно сварить и вегетарианский суп - на воде. Обычно я рассольник варю без томатной пасты, но тут для разнообразия решила добавить - это по желанию. Из специй соль, чёрный перец горошком, душистый перец.
Перловую крупу промыть до чистой воды.
В горячий бульон добавить промытую перловку и варить на среднем огне.
Для рассольника лучше брать кислые, очень солёные огурцы. Если же огурцы обычные, то рекомендуется в рассольник добавлять из сам рассол от огурцов. Солёные огурцы достать из рассола и натереть на крупной тёрке.
Картофель помыть, обсушить, очистить. Нарезать кубиками. Пока очередь картофеля не подошла, положить его в воду.
Морковь, лук, чеснок очистить. Морковь натереть на крупной тёрке, лук, сельдерей, чеснок мелко порезать.
Обжарить в масле овощи, добавив томатной пасты. Томатная паста по желанию.
Минут через двадцать добавить к перло

In [8]:
print(df_rec.loc[10, 'Инструкции'])

Приготовим ингредиенты по списку. Поскольку мы используем свежую капусту, а не квашенную, то должен быть какой-то продукт, который создаст кислый вкус, отличающий его от других супов. Это могут быть антоновские яблоки, щавель или соленые грибы. Я использовала последний из них.
Морковь моем, чистим и натираем на средней терке.
Репчатый лук чистим от шелухи и мелко нарезаем ножом.
Белокочанную капусту очищаем от верхних сухих листьев и мелко шинкуем ножом.
Картофель моем, чистим и нарезаем небольшими кубиками или соломкой, снова промываем, чтобы удалить крахмал и сделать бульон прозрачным. Картофель сделает суп гуще и сытнее.
Грибы можно использовать любые: шампиньоны или лесные, свежие, маринованные, соленые. Я использовала соленые рыжики, с их добавлением щи получились ароматными и с пикантной легкой остринкой и кислинкой, именно это является отличительной чертой супа - щей. Соленые грибы промываем, даем воде стечь, если грибочки крупные нарезаем их на маленькие кубики. Если используют

In [9]:
df_rec.shape


(4311, 10)

In [10]:
data = df_rec.loc[:5000, 'Инструкции']

In [11]:
import re
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [12]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')


In [13]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 3664
Test dataset length: 647


In [14]:
from transformers import AutoTokenizer
#sberbank-ai/rugpt3large_based_on_gpt2
#sberbank-ai/rugpt3medium_based_on_gpt2
#sberbank-ai/rugpt3small_based_on_gpt2

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)


/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [16]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [17]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [19]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5835
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 4377
  Number of trainable parameters = 125231616


Step,Training Loss
500,2.532100
1000,2.459100
1500,2.367900
2000,2.182000
2500,2.169000
3000,2.122500
3500,1.980800
4000,1.982200


Saving model checkpoint to ./gpt2-chief/checkpoint-800
Configuration saved in ./gpt2-chief/checkpoint-800/config.json
Configuration saved in ./gpt2-chief/checkpoint-800/generation_config.json
Model weights saved in ./gpt2-chief/checkpoint-800/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-1600
Configuration saved in ./gpt2-chief/checkpoint-1600/config.json
Configuration saved in ./gpt2-chief/checkpoint-1600/generation_config.json
Model weights saved in ./gpt2-chief/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-2400
Configuration saved in ./gpt2-chief/checkpoint-2400/config.json
Configuration saved in ./gpt2-chief/checkpoint-2400/generation_config.json
Model weights saved in ./gpt2-chief/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to ./gpt2-chief/checkpoint-3200
Configuration saved in ./gpt2-chief/checkpoint-3200/config.json
Configuration saved in ./gpt2-chief/checkpoint-3200/generation_config.json
Model weights 

TrainOutput(global_step=4377, training_loss=2.2034562938638764, metrics={'train_runtime': 837.6332, 'train_samples_per_second': 20.898, 'train_steps_per_second': 5.225, 'total_flos': 1143479255040000.0, 'train_loss': 2.2034562938638764, 'epoch': 3.0})

In [20]:
trainer.save_model()

Saving model checkpoint to ./gpt2-chief
Configuration saved in ./gpt2-chief/config.json
Configuration saved in ./gpt2-chief/generation_config.json
Model weights saved in ./gpt2-chief/pytorch_model.bin


In [21]:
tokenizer.save_pretrained('gpt_chf')

tokenizer config file saved in gpt_chf/tokenizer_config.json
Special tokens file saved in gpt_chf/special_tokens_map.json


('gpt_chf/tokenizer_config.json',
 'gpt_chf/special_tokens_map.json',
 'gpt_chf/vocab.json',
 'gpt_chf/merges.txt',
 'gpt_chf/added_tokens.json',
 'gpt_chf/tokenizer.json')

In [22]:
tokenizer.save_pretrained('gpt_chf')
model.save_pretrained('model_gpt_chf')

tokenizer config file saved in gpt_chf/tokenizer_config.json
Special tokens file saved in gpt_chf/special_tokens_map.json
Configuration saved in model_gpt_chf/config.json
Configuration saved in model_gpt_chf/generation_config.json
Model weights saved in model_gpt_chf/pytorch_model.bin


In [23]:
tokenizer = AutoTokenizer.from_pretrained("gpt_chf")
model1 = AutoModelForCausalLM.from_pretrained("model_gpt_chf")

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file model_gpt_chf/config.json
Model config GPT2Config {
  "_name_or_path": "model_gpt_chf",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype

In [24]:
prefix = "берем свежие томаты "

In [25]:
tokens = tokenizer(prefix, return_tensors='pt')
#tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [26]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+50, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


берем свежие томаты  Помидоры моем, чистим от кожицы и нарезаем небольшими кубиками. Обжариваем их на растительном масле в течение 2-3 минут с каждой стороны до золотистой корочки. Затем добавляем к помидорам нарез


In [27]:
prefix = "чистим баклажаны "

In [28]:
tokens = tokenizer(prefix, return_tensors='pt')

In [29]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+50, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


чистим баклажаны  Болгарский перец очистим от семян и перегородок, нарежем тонкими полукольцами. Обжарим его на растительном масле до золотистого цвета (примерно 2-3 минуты). Добавим в сковороду томатную па


In [30]:
prefix = "отвариваем яйца вкрутую "

In [31]:
tokens = tokenizer(prefix, return_tensors='pt')

In [32]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+50, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


отвариваем яйца вкрутую  3. В кипящий бульон добавляем нарезанный кубиками картофель и варим до готовности картофеля (около 10 минут)  4. Готовый суп разливаем по тарелкам, посыпаем мелко нарезанным укропом или петрушкой
